In [2]:
import numpy as np

In [8]:
class neuralNetwork():
    def __init__(self, layers, activation, output_activation='sigmoid', learning_rate = 0.1, epochs=100):
        self.layers = layers
        self.activation = activation
        self.outputActivation = output_activation
        self.epochs = epochs
        self.learningRate = learning_rate
        self.weights = []
        self.bias = []
        self.targets = []
    def train(self,x,y):
        self.initializeWeights(x.shape[1])
        self.initializeBias()
        for i in range(self.epochs):
            target = self.feedForward(x)
            self.backpropagate(y)
            mse = self.MSE(y,target)
            if i % 10 == 0:
                print('epoch: ',i)
                print('mse error: ', mse)
                print('====================')
    def pred(self,x):
        print(self.feedForward(x))
        
    def sigmoid(self,x):
        return 1 / (1 + np.exp(-x))
    def relu(self,x):
        return np.heaviside(x,1)*x
    def feedForward(self,x):
        self.targets = []
        self.targets.append(x)
        for i in range(len(self.weights)-1):
            if self.activation == 'sigmoid':
                self.targets.append(self.sigmoid(np.dot(self.targets[-1],self.weights[i]) + self.bias[i]))
            elif self.activation == 'relu':
                self.targets.append(self.relu(np.dot(self.targets[-1],self.weights[i]) + self.bias[i]))
            elif self.activation == 'linear':
                self.targets.append(np.dot(self.targets[-1],self.weights[i]) + self.bias[i])
        #output layer
        if self.outputActivation == 'sigmoid':
            self.targets.append(self.sigmoid(np.dot(self.targets[-1],self.weights[-1]) + self.bias[-1]))
        if self.outputActivation == 'linear':
            self.targets.append(np.dot(self.targets[-1],self.weights[-1]) + self.bias[-1])
        return self.targets[-1]
    def MSE(self,y,target):
        return ((target - y) ** 2).mean()
    def initializeWeights(self,xshape):
        self.weights.append(np.random.rand(xshape,self.layers[0]))
        if len(self.layers)>1:
            for i in range(1,len(self.layers)):
                weight = np.random.rand(self.layers[i-1],self.layers[i])
                self.weights.append(weight)
    def initializeBias(self):
        self.bias.append(np.ones(self.layers[0]))
        if len(self.layers)>1:
            for i in range(1,len(self.layers)):
                bias = np.ones(self.layers[i])
                self.bias.append(bias)

    def backpropagate(self,y):
        cellDelta = []
        oldWeights = self.weights[:]
        oldBias = self.bias[:]
        #output layer
        TargetLayerInput = np.dot(self.targets[-2],self.weights[-1]) + self.bias[-1]
        ##activation functions
        if self.outputActivation == 'sigmoid':
            df = self.sigmoid(TargetLayerInput)*(1-self.sigmoid(TargetLayerInput))
        elif self.outputActivation == 'relu':
            df = np.heaviside(TargetLayerInput,1)
        elif self.outputActivation == 'linear':
            df = np.ones(TargetLayerInput.shape)
        cellDelta.append(np.array((-2*(y-self.targets[-1])*df),ndmin = 2))
        for i in range(len(y)):
            dummy1 = np.array(self.targets[-2][i],ndmin = 2)
            dummy2 = np.array(cellDelta[-1][i],ndmin = 2)
            biasDummy = np.ones(self.bias[-1].shape)
            dw = dummy1.T@dummy2
            db = biasDummy*dummy2
            self.weights[-1] -= self.learningRate*dw
            self.bias[-1] -= self.learningRate*db[0]
        #hidden layers
        for i in range(len(self.layers)-2,-1,-1):
            layerInput = np.dot(self.targets[i],oldWeights[i]) + oldBias[i]
            if self.activation == 'sigmoid':
                df = self.sigmoid(layerInput)*(1-self.sigmoid(layerInput))
            elif self.activation == 'relu':
                df = np.heaviside(layerInput,1)
            elif self.activation == 'linear':
                df = layerInput
            dummyDelta = np.dot(cellDelta[-1],oldWeights[i+1].T)
            cellDelta.append(dummyDelta*df)
            for j in range(len(y)):
                dummy1 = np.array(self.targets[i][j],ndmin = 2)
                dummy2 = np.array(cellDelta[-1][j],ndmin = 2)
                biasDummy = np.ones(self.bias[i].shape)
                db = biasDummy*dummy2
                dw = dummy1.T@dummy2
                self.weights[i] -= self.learningRate*dw
                self.bias[i] -= self.learningRate*db[0]

In [9]:
model = neuralNetwork([2,1],'sigmoid','linear',learning_rate = 0.1, epochs = 100)

data = np.array([
  [-2, -1],  # Alice
  [25, 6],   # Bob
  [17, 4],   # Charlie
  [-15, -6], # Diana
])

all_y_trues = np.array([
  [1], # Alice
  [0], # Bob
  [0], # Charlie
  [1], # Diana
])

model.train(data,all_y_trues)

epoch:  0
mse error:  2.4939046204967013
epoch:  10
mse error:  0.050416557165164896
epoch:  20
mse error:  0.019692243994118314
epoch:  30
mse error:  0.012125022369666495
epoch:  40
mse error:  0.008606991213275995
epoch:  50
mse error:  0.006559540168477542
epoch:  60
mse error:  0.0052416531200026786
epoch:  70
mse error:  0.004333057034618873
epoch:  80
mse error:  0.003673782578334807
epoch:  90
mse error:  0.0031762172124220342
